In [64]:

import pandas as pd
import numpy as np

input = pd.read_csv('/content/drive/MyDrive/명륜이네 서재/DL_FINAL/new_input.csv')
student = pd.read_csv('/content/drive/MyDrive/명륜이네 서재/DL_FINAL/new_student_major.csv') #major/student 
book = pd.read_csv('/content/drive/MyDrive/명륜이네 서재/DL_FINAL/book_15.csv')

input.shape #(21966, 5)
input.head()
# student.shape #(21966, 2)
# student.head()
# book.shape #(133878, 5)

,booklist,ddclist,major,pred,student_id
0,"[22234, 38591, 38927, 44301, 58994, 59239, 621...",[0 3 8 0 8 0 6 8 6 0 3 8 6 8 8 8 8 8],사과대학,109047.0,412737.0
1,"[20164, 27532, 71562, 79470, 79607, 81349, 95259]",[0 8 1 8 8 1 3],소프트웨어대학,99850.0,572900.0
2,"[1004, 1148, 1716, 4074, 4414, 4807, 5056, 996...",[8 1 8 5 5 5 1 6 8 5 8 0 5 1 5 1 5 3 5 9 1 3 8],소프트웨어대학,131493.0,991018.0
3,"[41531, 45894, 54247, 56907, 73093, 90789]",[7 7 7 9 7 8],사과대학,128795.0,1424110.0
4,"[8604, 8605, 10238, 22870, 74592, 76788, 84120...",[3 3 2 8 1 9 1 8 5 8 8 8],사과대학,99339.0,1520139.0


### 1. PADD WITH MEDIAN VALUE 84

In [143]:
#padd the booklist 

from tensorflow.keras.preprocessing.sequence import pad_sequences

books = input['booklist'].tolist()
type(books[0])
books
new=[]
from ast import literal_eval
for x in books:
  x = literal_eval(x)
  new.append(x)

max_len = 84
new[:10]
pad_new = pad_sequences(new, maxlen = max_len)
pad_new.shape #(21966, 84)
pad_new[1]

#make pad_x for input 

np.save('pad_x.npy', pad_new)


In [144]:
new

[[22234,
  38591,
  38927,
  44301,
  58994,
  59239,
  62180,
  65120,
  66030,
  67437,
  69274,
  73285,
  86018,
  97021,
  100902,
  101232,
  101535,
  101768],
 [20164, 27532, 71562, 79470, 79607, 81349, 95259],
 [1004,
  1148,
  1716,
  4074,
  4414,
  4807,
  5056,
  9960,
  11092,
  12095,
  12463,
  15257,
  17631,
  20991,
  22420,
  23228,
  24685,
  26074,
  29376,
  60793,
  68922,
  79599,
  100317],
 [41531, 45894, 54247, 56907, 73093, 90789],
 [8604,
  8605,
  10238,
  22870,
  74592,
  76788,
  84120,
  89158,
  98269,
  98604,
  98606,
  98929],
 [39341,
  39342,
  39366,
  40801,
  51803,
  53171,
  53706,
  53782,
  55810,
  56607,
  61315,
  65151,
  68659,
  74244,
  75125,
  80442,
  82928,
  87037,
  87876],
 [6215, 6249, 12218, 12918, 31216, 69117, 80594, 84809, 95241],
 [40526,
  41340,
  52186,
  57564,
  57895,
  64424,
  69327,
  69863,
  77819,
  80509,
  83059,
  86810,
  94568],
 [1919, 6661, 8022, 8589, 19006, 19456, 28651, 31379, 74005, 101731],
 [80

### 2. RECREATION of Major EMBEDDINGS 

In [49]:
#student major embedding 
input['major'].value_counts()
input.shape

!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

bert_model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')


major_embedd = bert_model.encode(input['major'])
major_embedd.shape #(21966, 768)


np.save('major_embeddings.npy', major_embedd) 


In [61]:

input.shape
student.shape
book.shape

(133878, 5)

### 3. RECREATION OF Y(MULTI LABEL) 

In [69]:

import pickle
pkl_file = open('/content/drive/MyDrive/Colab Notebooks/Bookid_encoder.pkl', 'rb')
le = pickle.load(pkl_file) 
pkl_file.close()   
book_student = pd.read_csv('/content/drive/MyDrive/명륜이네 서재/FINAL_DATA/DATA_PREVIOUS_VERSION/dl_book_student_15.csv')
book_student

,student_id,major,book_id,ddc,title,ddc_short
0,412737,사과대학,EE0443182,001.3 ㅈ548ㅈ c.2,지금 시작하는 인문학,0
1,412737,사과대학,EM0375546,396 A544t7 c.3,性의 사회학,3
2,412737,사과대학,EM0401836,811 ㄱ971ㅎ c.4,한국문학의 위상,8
3,412737,사과대학,EM0571494,028 ㄷ233ㄴ c.4,나는 이런 책을 읽어왔다,0
4,412737,사과대학,EM0743789,811.36 ㅇ641ㅅ c.2,사서함 110호의 우편물,8
...,...,...,...,...,...,...
463353,9999547230,경제대학,EM0918504,306.6098 ㅂ313ㄹ,라틴아메리카 종교와 문화,3
463354,9999547230,경제대학,EM0920590,823.914 K35j8 c.3,더 잡,8
463355,9999547230,경제대학,EM0936671,823.914 O791n7 2014,1984,8
463356,9999547230,경제대학,EM0936899,869.35 C672a4 c.2,불륜,8


In [76]:
#label encode bookid 
le.inverse_transform([83973])

le.transform(['EM0891376'])
#bookid -> number encoded 

array([83973])

In [84]:
#1 recreate booklist 

#make the student book table 
from tqdm import tqdm
import time

#unique student id 
unique_id = student['student_id'].unique()
# len(unique_id) # students is 22024
df= pd.DataFrame()
readonlyonebook = []

for i in tqdm(unique_id):
  #look at unique id in book_student df 
  student_df = book_student[book_student['student_id'] == i ]
  books = student_df['book_id'].values
  ddcs = student_df['ddc_short'].values  
  major = student_df['major'].values[0]
  

  #change the books into label encoded nums 
  books = le.transform(books).astype(int)

    #remove the last ddc & book 
  # print(student_df.shape)


  #read more than 84 books 
  if student_df.shape[0] > 84 :
    ddcs = ddcs[:85] #include 84
    read = books[:85] 
    pred = books[85:] #starts from 85
    df =df.append({'student_id' : i, 
             'booklist' : list(read),
             'ddclist' : ddcs,
             'major': major,
             'pred' : list(pred)}, ignore_index=True)
  
  #when read less than 84 books 
  else:
    ddcs = ddcs[:-1]
    read = books[:-1]
    pred = books[-1]
    df =df.append({'student_id' : i, 
             'booklist' : list(read),
             'ddclist' : ddcs,
             'major': major,
             'pred' : pred}, ignore_index=True)
    
  


df.head()

100%|██████████| 21966/21966 [14:00<00:00, 26.13it/s]


,booklist,ddclist,major,pred,student_id
0,"[22234, 38591, 38927, 44301, 58994, 59239, 621...","[0, 3, 8, 0, 8, 0, 6, 8, 6, 0, 3, 8, 6, 8, 8, ...",사과대학,109047,412737.0
1,"[20164, 27532, 71562, 79470, 79607, 81349, 95259]","[0, 8, 1, 8, 8, 1, 3]",소프트웨어대학,99850,572900.0
2,"[1004, 1148, 1716, 4074, 4414, 4807, 5056, 996...","[8, 1, 8, 5, 5, 5, 1, 6, 8, 5, 8, 0, 5, 1, 5, ...",소프트웨어대학,131493,991018.0
3,"[41531, 45894, 54247, 56907, 73093, 90789]","[7, 7, 7, 9, 7, 8]",사과대학,128795,1424110.0
4,"[8604, 8605, 10238, 22870, 74592, 76788, 84120...","[3, 3, 2, 8, 1, 9, 1, 8, 5, 8, 8, 8]",사과대학,99339,1520139.0


In [90]:
df[500:600]

,booklist,ddclist,major,pred,student_id
500,"[40235, 43865, 47865, 52165, 59313, 66265, 678...","[3, 3, 1, 3, 3, 2, 3, 2, 8, 3, 6]",경제대학,129470,227088310.0
501,"[15221, 19266, 34436, 37496, 37932, 54995, 640...","[5, 5, 1, 3, 0, 1, 3, 5, 3, 3, 5, 5, 5, 0, 3, ...",경제대학,131771,227931340.0
502,"[13505, 14148, 15739, 16464, 23431, 28066, 305...","[3, 3, 3, 1, 8, 1, 9, 6, 8, 1, 6, 6, 3, 1, 3, ...",사과대학,132670,228015390.0
503,"[327, 724, 1057, 2764, 3530, 5906, 6131, 7603,...","[6, 8, 8, 7, 7, 7, 7, 7, 8, 9, 6, 7, 9, 9, 7, ...",공과대학,133204,230091249.0
504,"[4956, 17834, 38447, 52200, 66069, 73417, 7979...","[3, 3, 9, 3, 3, 3, 9, 1, 9]",경영대학,116318,230216620.0
...,...,...,...,...,...
595,"[3756, 9756, 14901, 15246, 27361, 27712, 51636...","[6, 6, 6, 6, 6, 6, 8, 5, 6, 6]",공과대학,133480,266700820.0
596,"[11091, 14156, 21085, 21488, 32447, 36422, 365...","[8, 8, 3, 8, 9, 8, 8, 1, 1, 8, 5, 5, 7, 5, 5, ...",경제대학,123457,266879280.0
597,"[1217, 2920, 17631, 22938, 31688, 34907, 36035...","[5, 5, 5, 5, 1, 1, 8, 3, 8, 8, 8, 8, 8, 1, 8, 8]",자연과학대학,120951,267182510.0
598,"[45982, 46240, 61820, 84388, 84769]","[8, 9, 4, 8, 1]",사범대학,88374,268492529.0


In [85]:
df.to_csv('second_input.csv', index=False)

In [ ]:
from scipy.sparse import csr_matrix 
  
# Creating a sparse matrix  #unique_student, unique_book
train_y = csr_matrix((input.shape[0], book.shape[0]), dtype = np.int8).toarray()  #학생수, 
train_y

y = df[['pred']] 
y.shape #(21966, 1)
# print(y.iloc[0])

train_y.shape #(21966, 133878)



for i in range(y.shape[0]):
  print(i)
  #when the length is longer than one = read more than 85 books 
  try: 
    len(y.iloc[i,0])
    indexs = y.iloc[i,0]
    for j in indexs: 
      print('j  '+str(j))
      train_y[i,j] = 1

  #has only one element 
  except:
    train_y[i,int(y.iloc[i,0])] = 1 
  # print(train_y[i,int(y.iloc[i,0])])
  
  



In [132]:
train_y[599]

# train_y.sum()

np.save('train_y.npy', train_y) 
train_y.shape

(21966, 133878)

#### 4. PAddings for second input 

In [147]:
#padd the booklist 

second_input = df
second_input.shape #(21966, 5)

from tensorflow.keras.preprocessing.sequence import pad_sequences
from ast import literal_eval



books = second_input['booklist'].tolist()
type(books[0])
books

pad_new = pad_sequences(books, maxlen = max_len)
pad_new.shape #(21966, 84)
# pad_new[1]
np.save('second_pad_x.npy', pad_new)


In [150]:
pad_new[599]

array([ 9170,  9171, 12412, 12929, 13991, 14142, 19423, 19504, 21380,
       22111, 22338, 23078, 23834, 25100, 25470, 25628, 26883, 26951,
       27292, 27887, 28648, 28760, 32124, 32619, 32895, 35441, 39795,
       39979, 40233, 40361, 40488, 40489, 41736, 41787, 42485, 43429,
       44013, 44754, 44755, 45151, 45655, 45818, 46213, 46753, 47871,
       48128, 48459, 48503, 52693, 53568, 54428, 55586, 55761, 56582,
       57025, 58880, 59174, 59480, 60049, 60805, 61005, 62837, 63154,
       65498, 66388, 66585, 66696, 68378, 69045, 69280, 69541, 70915,
       71814, 72263, 74673, 74897, 77406, 78227, 79374, 79604, 80725,
       81871, 82140, 83588], dtype=int32)